In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## Chargement des données

In [2]:
fafr = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Query_poly_20210222091846.xlsx", converters={'Matricule':str}) # effectif présent
infoc = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\individus_liste_info_complementaires_20210222093217.xlsx", converters={'Code de la ressource':str}) # données extenes
annuaire = pd.read_csv(r"C:\Users\sgasmi\Desktop\Extractions\Export_annuaire_20210222.csv", sep=";", encoding ="ANSI", low_memory=False, error_bad_lines=False) # Extraction de l'annuaire
indentité = pd.read_excel(r"C:\Users\sgasmi\Desktop\Interlocuteurs\Gestionnaires d'identité.xlsx")
excl_sal = pd.read_excel(r"C:\Users\sgasmi\Desktop\Interlocuteurs\Salariés à exclure.xlsx")

In [3]:
## Annuaire, transformation de clé "employeeNumber" en int

def to_int(x):
    try:
        x = int(x)
    except:
        x = 0    
    return x
    

annuaire["employeeNumber"] = annuaire["employeeNumber"].apply(lambda x: to_int(x))
annuaire = annuaire.loc[annuaire["employeeNumber"]!=0]

print(annuaire.shape)

(10676, 107)


In [4]:
excl_sal["Matricule"] = excl_sal["Matricule"].astype("int64")

## Contrôle des données

### Contrôle login SAP

In [5]:
fafr = fafr.merge(infoc[["Code de la ressource", "Login SAP"]], left_on=["Matricule"], right_on=["Code de la ressource"], how='left')

In [6]:
fafr["Login SAP vide"] = ""
fafr.loc[fafr["Login SAP"].isna()==True, "Login SAP vide"] = "Oui"
fafr_sap_ko = fafr.loc[fafr["Login SAP vide"]=="Oui"]
fafr_sap_ko = fafr_sap_ko.merge(indentité, on="Etablissement", how="left")
fafr_sap_ko = fafr_sap_ko.sort_values(by=["Etablissement"])
fafr_sap_ko["Matricule"] =fafr_sap_ko["Matricule"].astype("int64")

In [7]:
fafr_sap_ko = fafr_sap_ko.merge(annuaire[["employeeNumber", "saplogon"]],left_on="Matricule", right_on="employeeNumber", how="left")

In [8]:
fafr_sap_ko = fafr_sap_ko[["Matricule","Nom", "Prénom", "Etablissement","Gestionnaire d'identité", "employeeNumber", "saplogon"]]
fafr_sap_ko = fafr_sap_ko.loc[fafr_sap_ko["Matricule"].isin(excl_sal["Matricule"])==False]
sap_ko_annuaire = fafr_sap_ko.loc[fafr_sap_ko["saplogon"].isna()]

In [9]:
sap_ko_annuaire.loc[sap_ko_annuaire["employeeNumber"].isna()==True, "Causes possibles"] = "Matricule différent de celui du fichier/ Salarié Tiers au lieu d'agent/ Salarié non rattaché à Framatome"
sap_ko_annuaire.loc[sap_ko_annuaire["employeeNumber"].isna()==False, "Causes possibles"] = "Login SAP du compte Agent non renseigné/ Salarié non rattaché à FRAMATOME"

C:\Users\sgasmi\Miniconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\sgasmi\Miniconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\sgasmi\Miniconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

## Export

### Export login SAP KO

In [10]:
with pd.ExcelWriter(r'C:\Users\sgasmi\Desktop\Export Python\SAP_KO.xlsx') as writer:  
    fafr_sap_ko.to_excel(writer, sheet_name='Conso globale', index=False)
    sap_ko_annuaire.to_excel(writer, sheet_name='SAP Manquant dans annuaire', index=False)